In [1]:
import pandas as pd
import datetime
import vectorbt as vbt
import numpy as np
import matplotlib.pyplot as plt
import vbt_indicator
import typing

from importlib import reload
reload(vbt_indicator)
from vbt_indicator import *

c:\Users\slavi\anaconda3\envs\Bruno_proj\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [28]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 10)  #temps de download d'environ 30s / crtypto pour 10 jours de données en 1mn pour moi
interval = '1d'
list_assets = ['BTCUSDT', 'ETHUSDT']  #Rajouter les cryptos à tester
price = vbt.BinanceData.download(list_assets,
                                interval = interval,
                                start = start_date,
                                end = end_date,
                                missing_index= 'drop').get([ 'Close', 'Open'])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [29]:
price

(symbol                      BTCUSDT  ETHUSDT
 Open time                                   
 2023-05-12 00:00:00+00:00  26795.01  1807.45
 2023-05-13 00:00:00+00:00  26775.28  1795.11
 2023-05-14 00:00:00+00:00  26917.62  1799.21
 2023-05-15 00:00:00+00:00  27162.14  1816.20
 2023-05-16 00:00:00+00:00  27033.84  1824.21
 2023-05-17 00:00:00+00:00  27405.61  1822.12
 2023-05-18 00:00:00+00:00  26821.28  1800.56
 2023-05-19 00:00:00+00:00  26880.26  1812.32
 2023-05-20 00:00:00+00:00  27102.43  1819.42
 2023-05-21 00:00:00+00:00  26737.45  1799.90,
 symbol                      BTCUSDT  ETHUSDT
 Open time                                   
 2023-05-12 00:00:00+00:00  26968.61  1794.31
 2023-05-13 00:00:00+00:00  26795.01  1807.45
 2023-05-14 00:00:00+00:00  26775.27  1795.12
 2023-05-15 00:00:00+00:00  26917.61  1799.20
 2023-05-16 00:00:00+00:00  27162.15  1816.21
 2023-05-17 00:00:00+00:00  27033.85  1824.21
 2023-05-18 00:00:00+00:00  27405.62  1822.12
 2023-05-19 00:00:00+00:00  26821

In [ ]:
class DataSpec:
    def __init__(self, 
                 symbols: typing.Union[typing.List[str], str],
                 interval : str, 
                 missing_index : str = 'drop', 
                 to_get : typing.Optional[typing.List[str]] = None):
        self.symbols = symbols
        self.interval = interval
        self.missing_index = missing_index
        self.to_get = to_get


In [ ]:
class StratSpec :
    def __init__(self, 
                 computeIndicator, 
                 computeEntries, 
                 computeExits, 
                 indicatorParams,
                 indicatorInputs,
                 entriesInputs,
                 exitsInputs):
        self.computeIndicator = computeIndicator
        self.computeEntries = computeEntries
        self.computeExits = computeExits
        self.indicatorsParams = indicatorParams
        self.indcatorInputs = indicatorInputs
        self.entriesInputs = entriesInputs
        self.exitsInputs = exitsInputs
    

In [16]:
class ForwardBacktest:

    def __init__(self, startBacktest : np.datetime64, 
                 endBacktest : np.datetime64, 
                 dataSpec : DataSpec,
                 stratSpec : StratSpec,
                 optimizingWindow : np.timedelta64 = np.timedelta64(2, 'M'),
                 runingWindow : np.timedelta64 = np.timedelta64(1, 'M'), 
                 dataInstance : vbt.data.base.Metadata = vbt.BinanceData):

        self.__startBacktest = startBacktest
        self.__endBacktest = endBacktest
        self.__dataInstance = dataInstance
        self.__dataSpec = dataSpec
        self.__stratSpec = stratSpec
        self.__optimizingWindow = optimizingWindow
        self.__runingWindow = runingWindow


    
    def _getPrice(self, start, end):
        return self.__dataInstance.download(symbols = self.__dataSpec.symbols,
                                            interval = self.__dataSpec.interval,
                                            start = start,
                                            end = end, 
                                            missing_index = self.__dataSpec.missing_index ).get(self.__dataSpec.to_get)
    

    def _localOptim(self, start, end): 
        
